<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/GraphSAGE_BANGLA_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 39.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 20.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.1 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-3famf491
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-3famf491
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 241a8c3d018636c116fd1fd7fa2ab9ff3925531e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.6.0-py3-none-any.whl size=11

In [3]:
!pip install networkx
# Step 2: Import Libraries
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import pandas as pd
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# Step 3: Upload Dataset
from google.colab import files
uploaded = files.upload()  # Upload your CSV file (e.g., dataset.csv)

# Load the dataset
df = pd.read_excel(pd.ExcelFile(list(uploaded.keys())[0]), header=0) # Expecting two columns: 'tweets' and 'labels' where label is 0 or 1
df = df.sample(frac=0.4, random_state=42)
df = df[['tweets', 'labels']]

# Step 4: Preprocess Text Data and Create Embeddings
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModel.from_pretrained('xlm-roberta-base')




def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Generate embeddings for all texts
embeddings = get_embeddings(df['tweets'].tolist())
labels = torch.tensor(df['labels'].values, dtype=torch.long)

# Step 5: Create Graph Using NetworkX
G = nx.Graph()
# Add nodes with features
for i in range(len(df)):
    G.add_node(i, x=embeddings[i])

# Step 6: Create Edges Using Cosine Similarity
similarity_matrix = cosine_similarity(embeddings)
threshold = 0.8  # Define a threshold for creating edges

# Add edges based on similarity
for i in range(len(similarity_matrix)):
    for j in range(i + 1, len(similarity_matrix)):
        if similarity_matrix[i, j] > threshold:
            G.add_edge(i, j)

# Convert NetworkX graph to PyTorch Geometric Data
data = from_networkx(G)
data.x = torch.stack([data.x[i] for i in range(len(G.nodes))])
data.y = labels




Saving Bangla.xlsx to Bangla.xlsx


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [4]:
labeled_idx, test_idx = train_test_split(range(len(df)), test_size=0.2, stratify=df['labels'])
labeled_idx, unlabeled_idx = train_test_split(labeled_idx, test_size=0.75, stratify=df['labels'].iloc[labeled_idx])

data.labeled_mask = torch.tensor(labeled_idx, dtype=torch.long)
data.unlabeled_mask = torch.tensor(unlabeled_idx, dtype=torch.long)
data.test_mask = torch.tensor(test_idx, dtype=torch.long)


In [5]:

# Step 8: Define GraphSAGE Model
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the Student and Teacher Models
student_model = GraphSAGE(in_channels=embeddings.size(1), hidden_channels=128, out_channels=2)
teacher_model = GraphSAGE(in_channels=embeddings.size(1), hidden_channels=128, out_channels=2)
optimizer = torch.optim.Adam(student_model.parameters(), lr=0.01)

# EMA Update Function for Teacher Model
def update_teacher_model(teacher, student, alpha=0.99):
    for teacher_param, student_param in zip(teacher.parameters(), student.parameters()):
        teacher_param.data.mul_(alpha).add_(1 - alpha, student_param.data)

# Step 9: Training Functions
def train_student():
    student_model.train()
    optimizer.zero_grad()
    out = student_model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.labeled_mask], data.y[data.labeled_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def train_with_consistency_loss():
    student_model.eval()
    teacher_model.eval()

    # Predictions from student and teacher
    student_out = student_model(data.x, data.edge_index)
    teacher_out = teacher_model(data.x, data.edge_index)

    # Calculate consistency loss on unlabeled data
    student_probs = F.softmax(student_out[data.unlabeled_mask], dim=1)
    teacher_probs = F.softmax(teacher_out[data.unlabeled_mask], dim=1)
    consistency_loss = F.mse_loss(student_probs, teacher_probs)

    # Train the student with consistency loss
    student_model.train()
    optimizer.zero_grad()
    consistency_loss.backward()
    optimizer.step()
    return consistency_loss.item()

# Step 10: Test the Model
def test():
    teacher_model.eval()
    out = teacher_model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_pred = pred[data.test_mask]
    test_true = data.y[data.test_mask]
    acc = accuracy_score(test_true.cpu(), test_pred.cpu())
    f1 = f1_score(test_true.cpu(), test_pred.cpu())
    precision = precision_score(test_true.cpu(), test_pred.cpu())
    recall = recall_score(test_true.cpu(), test_pred.cpu())
    return acc, f1, precision, recall


In [6]:

# Train for a number of epochs
for epoch in range(1, 101):
    labeled_loss = train_student()
    consistency_loss = train_with_consistency_loss()
    update_teacher_model(teacher_model, student_model)

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Labeled Loss: {labeled_loss:.4f}, Consistency Loss: {consistency_loss:.4f}')

# Evaluate the model
acc, f1, precision, recall = test()
print(f'Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')


<ipython-input-5-319337e8e63f>:22: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1581.)
  teacher_param.data.mul_(alpha).add_(1 - alpha, student_param.data)


Epoch 10, Labeled Loss: 0.6911, Consistency Loss: 0.0052
Epoch 20, Labeled Loss: 0.6840, Consistency Loss: 0.0017
Epoch 30, Labeled Loss: 0.6696, Consistency Loss: 0.0024
Epoch 40, Labeled Loss: 0.6400, Consistency Loss: 0.0055
Epoch 50, Labeled Loss: 0.5956, Consistency Loss: 0.0108
Epoch 60, Labeled Loss: 0.5436, Consistency Loss: 0.0173
Epoch 70, Labeled Loss: 0.4918, Consistency Loss: 0.0239
Epoch 80, Labeled Loss: 0.4426, Consistency Loss: 0.0305
Epoch 90, Labeled Loss: 0.3987, Consistency Loss: 0.0362
Epoch 100, Labeled Loss: 0.3515, Consistency Loss: 0.0405
Accuracy: 0.5886, F1 Score: 0.4800, Precision: 0.6977, Recall: 0.3659


In [8]:
def test():
    student_model.eval()
    out = student_model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_pred = pred[data.test_mask]
    test_true = data.y[data.test_mask]
    acc = accuracy_score(test_true.cpu(), test_pred.cpu())
    f1 = f1_score(test_true.cpu(), test_pred.cpu())
    precision = precision_score(test_true.cpu(), test_pred.cpu())
    recall = recall_score(test_true.cpu(), test_pred.cpu())
    return acc, f1, precision, recall

In [9]:
acc, f1, precision, recall = test()
print(f'Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}')


Accuracy: 0.6899, F1 Score: 0.7200, Precision: 0.6774, Recall: 0.7683
